# Spotify Music Recommendation system

Algorithm : Content Based Filtering & Hybrid filtering

Distance method - Cosine Similarity

Notebook summary

1. Import necessary libraries

2. Import required dataset

3. Data preprocessing for model development

4. Building the recommendation engine model 1 - Content based filtering

5. Building the recommendation engine model 2 - Hybrid Recommendation Engine


# 1. Import necessary libraries

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

# 2. Import required dataset

In [37]:
music_df = pd.read_csv("music.csv")

In [38]:
music_df.shape

(100, 21)

In [39]:
music_df.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,I'm Good (Blue),"David Guetta, Bebe Rexha",I'm Good (Blue),7M842DMhYVALrXsw3ty7B3,4uUG5RXrOk84mYEfFvj3cK,94,2022-08-26,175238,True,https://open.spotify.com/track/4uUG5RXrOk84mYE...,...,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040
1,FE!N (feat. Playboi Carti),"Travis Scott, Playboi Carti",UTOPIA,18NOKLkZETa4sWwLMIm0UZ,42VsgItocQwOQC3XWZ8JNA,93,2023-07-28,191701,True,https://open.spotify.com/track/42VsgItocQwOQC3...,...,0.882,3,-2.777,0,0.0600,0.03160,0.000000,0.1420,0.201,148.038
2,Boy's a Liar Pt. 2,"PinkPantheress, Ice Spice",Boy's a liar Pt. 2,6cVfHBcp3AdpYY0bBglkLN,6AQbmUe0Qwf5PZnt4HmTXv,92,2023-02-03,131013,True,https://open.spotify.com/track/6AQbmUe0Qwf5PZn...,...,0.809,5,-8.254,1,0.0500,0.25200,0.000128,0.2480,0.857,132.962
3,"Quevedo: Bzrp Music Sessions, Vol. 52","Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",4PNqWiJAfjj32hVvlchV5u,2tTmW7RDtMQtBk7m2rYeSw,92,2022-07-06,198938,False,https://open.spotify.com/track/2tTmW7RDtMQtBk7...,...,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033
4,Me Porto Bonito,"Bad Bunny, Chencho Corleone",Un Verano Sin Ti,3RQQmkQEvNCY4prGKE6oc5,6Sq7ltF9Qa7SNFBsV5Cogx,91,2022-05-06,178567,True,https://open.spotify.com/track/6Sq7ltF9Qa7SNFB...,...,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005


In [40]:
music_df.shape

(100, 21)

In [41]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        100 non-null    object 
 1   Artists           100 non-null    object 
 2   Album Name        100 non-null    object 
 3   Album ID          100 non-null    object 
 4   Track ID          100 non-null    object 
 5   Popularity        100 non-null    int64  
 6   Release Date      100 non-null    object 
 7   Duration (ms)     100 non-null    int64  
 8   Explicit          100 non-null    bool   
 9   External URLs     100 non-null    object 
 10  Danceability      100 non-null    float64
 11  Energy            100 non-null    float64
 12  Key               100 non-null    int64  
 13  Loudness          100 non-null    float64
 14  Mode              100 non-null    int64  
 15  Speechiness       100 non-null    float64
 16  Acousticness      100 non-null    float64
 17

# 3. Data Preprocessing for model development

In [42]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability',
                           'Energy',
                           'Key',
                           'Loudness',
                           'Mode',
                           'Speechiness',
                           'Acousticness',
                           'Instrumentalness',
                           'Liveness',
                           'Valence',
                           'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [43]:
pd.DataFrame(music_features_scaled)

,0,1,2,3,4,5,6,7,8,9,10
0,0.214,0.988014,0.636364,0.741341,0.0,0.007605,0.004103,0.000008,0.443263,0.284413,0.568051
1,0.230,0.845890,0.272727,0.814706,0.0,0.063451,0.036443,0.000000,0.138985,0.174076,0.738291
2,0.484,0.720890,0.454545,0.366249,1.0,0.041721,0.293112,0.000142,0.279830,0.876808,0.609951
3,0.334,0.674658,0.181818,0.587816,1.0,0.028683,0.014199,0.036626,0.255913,0.547938,0.567992
4,0.914,0.554795,0.090909,0.624089,0.0,0.110604,0.104569,0.000030,0.074276,0.414033,0.261292
...,...,...,...,...,...,...,...,...,...,...,...
95,0.748,0.864726,0.090909,0.768280,1.0,0.040852,0.038073,0.005938,0.058065,0.603642,0.534026
96,0.678,0.595890,0.636364,0.775649,0.0,0.034985,0.095369,0.000000,0.095137,0.661489,0.439959
97,0.496,0.748288,0.545455,0.732007,0.0,0.063668,0.009926,0.000000,0.039862,0.938939,0.354695
98,0.514,0.381849,0.090909,0.595595,1.0,0.647979,0.006339,0.000000,0.257242,0.113016,0.619980


# 4. Building the recommendation engine model 1 - Content based filtering

### a. Defining the Recommendation Engine helper function

In [44]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]],
                                          music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name',
                                                                         'Artists',
                                                                         'Album Name',
                                                                         'Release Date',
                                                                         'Popularity']]

    return content_based_recommendations

### b. Testing the Recommendation Engine Function

Example 1

In [45]:
content_based_recommendations("I'm Good (Blue)" , 10)

,Track Name,Artists,Album Name,Release Date,Popularity
49,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,82
74,Where You Are,"John Summit, Hayla",Where You Are,2023-03-03,79
1,FE!N (feat. Playboi Carti),"Travis Scott, Playboi Carti",UTOPIA,2023-07-28,93
41,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,83
87,Rainfall (Praise You),Tom Santa,Rainfall (Praise You),2022-02-18,77
66,Where Did You Go? (feat. MNEK),"Jax Jones, MNEK",Where Did You Go (feat. MNEK),2022-02-04,80
90,Kernkraft 400 (A Better Day),"Topic, A7S",Kernkraft 400 (A Better Day),2022-06-17,77
50,The Motto,"Tiësto, Ava Max",The Motto,2021-11-04,82
25,Bad Memories (feat. Elley Duhé & FAST BOY),"MEDUZA, James Carter, Elley Duhé, FAST BOY",Bad Memories (feat. Elley Duhé & FAST BOY),2022-07-22,85
19,Lay Low,Tiësto,Lay Low,2023-01-06,86


Example 2

In [46]:
content_based_recommendations("Me Porto Bonito")

,Track Name,Artists,Album Name,Release Date,Popularity
54,Private Landing (feat. Justin Bieber & Future),"Don Toliver, Justin Bieber, Future",Love Sick,2023-02-24,81
63,Move Your Body,"Öwnboss, Sevek",Move Your Body,2021-10-29,81
22,Give It To Me - Full Vocal Mix,Matt Sassari,Give It To Me (Full Vocal Mix),2021-10-22,86
88,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),2022-05-13,77
27,Ferrari,"James Hype, Miggy Dela Rosa",Ferrari,2022-04-01,85


Example 3

In [47]:
content_based_recommendations("Quevedo: Bzrp Music Sessions, Vol. 52" , 10)

,Track Name,Artists,Album Name,Release Date,Popularity
71,Come & Go (with Marshmello),"Juice WRLD, Marshmello",Legends Never Die,2020-07-10,79
43,Deep Down (feat. Never Dull),"Alok, Ella Eyre, Kenny Dope, Never Dull",Deep Down (feat. Never Dull),2022-06-17,83
84,When I’m Gone (with Katy Perry),"Alesso, Katy Perry",When I’m Gone (with Katy Perry),2021-12-29,78
57,Thunder,"Gabry Ponte, LUM!X, Prezioso",Thunder,2021-05-07,81
95,Marianela (Que Pasa),"HUGEL, Merk & Kremont, Lirico En La Casa",Marianela (Que Pasa),2022-11-25,76
9,Save Your Tears,The Weeknd,After Hours,2020-03-20,89
56,Esta Vida,"Marshmello, Farruko",Esta Vida,2023-04-13,81
59,21 Reasons (feat. Ella Henderson),"Nathan Dawe, Ella Henderson",21 Reasons (feat. Ella Henderson),2022-04-29,81
34,Belly Dancer,"Imanbek, BYOR",Belly Dancer,2022-02-18,84
2,Boy's a Liar Pt. 2,"PinkPantheress, Ice Spice",Boy's a liar Pt. 2,2023-02-03,92


Example 4

In [48]:
content_based_recommendations("Lionheart (Fearless)")

,Track Name,Artists,Album Name,Release Date,Popularity
5,Baby Don't Hurt Me,"David Guetta, Anne-Marie, Coi Leray",Baby Don't Hurt Me,2023-04-06,93
77,Remember,"Becky Hill, David Guetta",Remember,2021-06-18,79
15,Murder In My Mind,Kordhell,Murder In My Mind,2022-01-21,88
86,One in a Million,"Bebe Rexha, David Guetta",One in a Million,2023-08-04,77
28,Pepas,Farruko,Pepas,2021-06-24,85


Example 5

In [49]:
content_based_recommendations("Levitating (feat. DaBaby)")

,Track Name,Artists,Album Name,Release Date,Popularity
25,Bad Memories (feat. Elley Duhé & FAST BOY),"MEDUZA, James Carter, Elley Duhé, FAST BOY",Bad Memories (feat. Elley Duhé & FAST BOY),2022-07-22,85
96,PLAYA DEL INGLÉS,"Quevedo, Myke Towers",PLAYA DEL INGLÉS,2022-12-15,76
6,El Merengue,"Marshmello, Manuel Turizo",El Merengue,2023-03-03,91
40,Mood (feat. iann dior),"24kGoldn, iann dior",El Dorado,2021-03-26,83
50,The Motto,"Tiësto, Ava Max",The Motto,2021-11-04,82


Example 6

In [50]:
content_based_recommendations("Channa Mereya")

'Channa Mereya' not found in the dataset. Please enter a valid song name.


# 5. Building the recommendation engine model 2 - Hybrid Recommendation Engine



### a. Defining the helper function to calculate weighted popularity score

In [51]:
# Function to calculate weighted popularity scores based on release date

def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span
    #(e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

### b. Defining the helper function for Hybrid recommendation engine

In [52]:
# a function to get hybrid recommendations based on weighted popularity


def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name,
                                                      num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name,
                                    'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name,
                                                                                              'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    weighted_pop_df = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, weighted_pop_df], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity',
                                                                ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

### Testing the Recommendation Engine

In [53]:
hybrid_recommendations("Levitating (feat. DaBaby)",10)

,Track Name,Artists,Album Name,Release Date,Popularity
2,El Merengue,"Marshmello, Manuel Turizo",El Merengue,2023-03-03,91.0
0,Bad Memories (feat. Elley Duhé & FAST BOY),"MEDUZA, James Carter, Elley Duhé, FAST BOY",Bad Memories (feat. Elley Duhé & FAST BOY),2022-07-22,85.0
3,Mood (feat. iann dior),"24kGoldn, iann dior",El Dorado,2021-03-26,83.0
5,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,83.0
4,The Motto,"Tiësto, Ava Max",The Motto,2021-11-04,82.0
6,La Llevo Al Cielo (Ft. Ñengo Flow),"Chris Jedi, Anuel AA, Chencho Corleone, Ñengo ...",La Llevo Al Cielo (Ft. Ñengo Flow),2022-05-20,80.0
9,LOKERA,"Rauw Alejandro, Lyanno, Brray",LOKERA,2022-07-25,80.0
7,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),2022-05-13,77.0
8,Rainfall (Praise You),Tom Santa,Rainfall (Praise You),2022-02-18,77.0
1,PLAYA DEL INGLÉS,"Quevedo, Myke Towers",PLAYA DEL INGLÉS,2022-12-15,76.0


In [54]:
hybrid_recommendations("Lionheart (Fearless)")

,Track Name,Artists,Album Name,Release Date,Popularity
0,Baby Don't Hurt Me,"David Guetta, Anne-Marie, Coi Leray",Baby Don't Hurt Me,2023-04-06,93.0
2,Murder In My Mind,Kordhell,Murder In My Mind,2022-01-21,88.0
4,Pepas,Farruko,Pepas,2021-06-24,85.0
1,Remember,"Becky Hill, David Guetta",Remember,2021-06-18,79.0
3,One in a Million,"Bebe Rexha, David Guetta",One in a Million,2023-08-04,77.0


In [55]:
hybrid_recommendations("Quevedo: Bzrp Music Sessions, Vol. 52")

,Track Name,Artists,Album Name,Release Date,Popularity
1,Deep Down (feat. Never Dull),"Alok, Ella Eyre, Kenny Dope, Never Dull",Deep Down (feat. Never Dull),2022-06-17,83.0
3,Thunder,"Gabry Ponte, LUM!X, Prezioso",Thunder,2021-05-07,81.0
0,Come & Go (with Marshmello),"Juice WRLD, Marshmello",Legends Never Die,2020-07-10,79.0
2,When I’m Gone (with Katy Perry),"Alesso, Katy Perry",When I’m Gone (with Katy Perry),2021-12-29,78.0
4,Marianela (Que Pasa),"HUGEL, Merk & Kremont, Lirico En La Casa",Marianela (Que Pasa),2022-11-25,76.0


In [56]:
hybrid_recommendations("Me Porto Bonito")

,Track Name,Artists,Album Name,Release Date,Popularity
2,Give It To Me - Full Vocal Mix,Matt Sassari,Give It To Me (Full Vocal Mix),2021-10-22,86.0
4,Ferrari,"James Hype, Miggy Dela Rosa",Ferrari,2022-04-01,85.0
0,Private Landing (feat. Justin Bieber & Future),"Don Toliver, Justin Bieber, Future",Love Sick,2023-02-24,81.0
1,Move Your Body,"Öwnboss, Sevek",Move Your Body,2021-10-29,81.0
3,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),2022-05-13,77.0


In [57]:
hybrid_recommendations("Channa Mereya")

'Channa Mereya' not found in the dataset. Please enter a valid song name.
